#### Bring in Required Libraries

In [1]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from icecream import ic
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [2]:
# ODBC Connection for FileMaker
start = timer()
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cnxn.timeout = 60
cursor_fm = cnxn.cursor()
print("FileMaker Connect Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Connect Time = 10.11 sec


In [2]:
# SQLAlchemy connection for SQL Server
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
# conn = engine.raw_connection()
conn_sql = engine.connect()

In [3]:
# ODBC Connection for AS400
CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""
dbcnxn = pyodbc.connect(CONNAS400)
cursor_as400 = dbcnxn.cursor()

In [4]:
sql_spares = """
    SELECT PROD.FPSPRMAST1.SPH_PART,
        STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
        STRIP(PROD.FPSPRMAST1.SPH_DESC1),
        STRIP(PROD.FPSPRMAST1.SPH_DESC2),
        STRIP(PROD.FPSPRMAST1.SPH_MFG),
        STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
        STRIP(PROD.FPSPRMAST2.SPD_CABINT),
        STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
        PROD.FPSPRMAST2.SPD_QOHCUR,
        PROD.FPSPRMAST1.SPH_CURSTD,
        STRIP(PROD.FPSPRMAST2.SPD_REODTE),
        STRIP(PROD.FPSPRMAST2.SPD_USECC),
        STRIP(PROD.FPSPRMAST2.SPD_PURCC),
        STRIP(PROD.FPSPRMAST2.SPD_QREORD)
    FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
    WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))
"""

In [6]:
sql_parts = """
    SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
        "Screw Part Number" AS Screw, "Band Feed from Band data",
        "Ship Diam Max", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
        "CameraInspectionRequired", "ScrDrvChk"
    FROM tbl8Tridon
"""

In [7]:
sql_bands = """
    SELECT "Band Part Number", "Feed Length","CutoutA Tool Number","CutoutB Tool Number","CutoutC Tool Number",
        "Dim A", "Dim B","Dim C", "Process", "Description"
    FROM BANDS
"""

### Execute Query on AS400

In [5]:
start = timer()
cursor_as400.execute(sql_spares)
result_spares = cursor_as400.fetchall()
print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

AS400 Connect/Query Time = 2.71 sec


#### Execute Query on SQL Server

In [97]:
# Clamp Data
start = timer()
cursor_fm.execute(sql_parts)
result_clamp = cursor_fm.fetchall()
print("FileMaker Clamp Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Clamp Query Time = 3.078 sec


In [8]:
# Band Data
start = timer()
cursor_fm.execute(sql_bands)
result_band = cursor_fm.fetchall()
print("FileMaker Band Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Band Query Time = 12.055 sec


#### Build Dataframe for Spare Parts

In [92]:
data_type_dict = {'StandardCost' : float, 'OnHand' : int, 'PartNum' : str, 'ReOrderPt' : int, 'ReOrderDate' : int}
df_spares = pd.DataFrame.from_records(result_spares)
df_spares.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand', 'StandardCost','ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
df_spares = df_spares.dropna()
df_spares = df_spares.astype(data_type_dict)
df_spares = df_spares.convert_dtypes()

df_obs = df_spares[df_spares.Cabinet.str.contains('OBS')]
df_obs_yest = pd.read_csv('c:\\temp\yesterday_obs.csv',header=None, sep='\t')
df_obs_yest.reset_index(drop=True, inplace=True)
df_obs.reset_index(drop=True, inplace=True)
df_obs_yest = df_obs_yest.fillna("")

df_obs_yest.columns = df_spares.columns
df_obs_yest = df_obs_yest.astype(data_type_dict)
df_obs_yest = df_obs_yest.convert_dtypes()

if not df_obs['PartNum'].equals(df_obs_yest['PartNum']):
    df_diff = pd.concat([df_obs, df_obs_yest]).drop_duplicates(keep=False)
    df_obs.to_csv('c:\\temp\yesterday_obs.csv', header=False, index=False, sep='\t')
    i = 0
    item_list = []
    for index, row in df_diff.iterrows():
        #print(row['PartNum'], row['EngPartNum'], row['Desc1'])
        item_list.append('<h5>Item ' + str(i) + '</h5>'
                             + '<p style="margin-left: 40px">'
                             + 'Part Number: ' + row['PartNum']
                             + '<br>Eng Part Number: <strong>' + row['EngPartNum'] + '</strong>'
                             + '<br>Description 1: ' + row['Desc1']
                             + '<br>Description 2: ' + row['Desc2']
                             + '<br>Manufacturer: ' + row['Mfg']
                             + '<br>Manufacturer Pn: <strong>' + row['MfgPn'] + '</strong>'
                             + '<br>Dept Use: ' + row['DeptUse']
                             + '<br>Dept Purch: ' + row['DeptPurch']
                             + '</p><br>')
        i += 1
    for item in item_list:
        print(item)  


<h5>Item 0</h5><p style="margin-left: 40px">Part Number: 1021446<br>Eng Part Number: <strong>091021446</strong><br>Description 1: PUNCH SUPPORT<br>Description 2: OBS ECR 14755<br>Manufacturer: <br>Manufacturer Pn: <strong></strong><br>Dept Use: 0750<br>Dept Purch: 0750</p><br>
<h5>Item 1</h5><p style="margin-left: 40px">Part Number: 1021447<br>Eng Part Number: <strong>091021447</strong><br>Description 1: OBS ECR 14755<br>Description 2: ****<br>Manufacturer: <br>Manufacturer Pn: <strong></strong><br>Dept Use: 0750<br>Dept Purch: 0750</p><br>
<h5>Item 2</h5><p style="margin-left: 40px">Part Number: 1021449<br>Eng Part Number: <strong>091021449</strong><br>Description 1: FINGER-<br>Description 2: ****<br>Manufacturer: <br>Manufacturer Pn: <strong></strong><br>Dept Use: 0750<br>Dept Purch: 0750</p><br>
<h5>Item 3</h5><p style="margin-left: 40px">Part Number: 1021450<br>Eng Part Number: <strong>091021450</strong><br>Description 1: FINGER<br>Description 2: <br>Manufacturer: <br>Manufacturer 

#### Build Dataframe for Clamp Data

In [175]:
df_clamps = pd.DataFrame.from_records(result_clamp)
# Set Column Names
df_clamps.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Feed', 'DiaMax', 'DiaMin', 'HexSz', 'BandThickness','BandWidth', 'CamInspect', 'ScrDrvChk']
# Set Data Types
data_type_dict = {'PartNumber' : str, 'Band' : str,'Housing' : str,'Screw' : str, 'Feed' : float, 'DiaMax' : float,'DiaMin' : float,'HexSz' : str,'BandThickness' : float, 'BandWidth' : float,'CamInspect' : str,'ScrDrvChk' : str}

# Due Some Filtering and Data Cleansing
df_clamps = df_clamps[df_clamps.Feed != 'N/A']
df_clamps = df_clamps[1:]
df_clamps = df_clamps.astype(data_type_dict)
df_clamps['CamInspect'] = df_clamps['CamInspect'].str.upper()
df_clamps['ScrDrvChk'] = df_clamps['ScrDrvChk'].str.upper()
df_clamps['Feed'] = df_clamps['Feed'].round(3)
df_clamps['BandWidth'] = df_clamps['BandWidth'].round(3)
df_clamps['BandThickness'] = df_clamps['BandThickness'].round(3)
df_clamps.fillna({'DiaMax' : 0.0}, inplace=True)
df_clamps.fillna({'DiaMin' : 0.0}, inplace=True)
df_clamps['DiaMax'] = df_clamps['DiaMax'].round(3)
df_clamps['DiaMin'] = df_clamps['DiaMin'].round(3)
df_clamps = df_clamps.dropna()
df_clamps = df_clamps.convert_dtypes()
df_clamps.sample(20)



,PartNumber,Band,Housing,Screw,Feed,DiaMax,DiaMin,HexSz,BandThickness,BandWidth,CamInspect,ScrDrvChk
146,M631096717,2345096,2351032,1278001,21.39,0.0,0.0,"5/16""",0.022,0.5,NO,NO
3674,5F10599,2105103,1151002,1278001MC,22.89,0.0,0.0,"5/16""",0.028,0.562,NO,NO
3509,6713552,2752013,2351044,1318001,5.27,0.0,0.0,"5/16""",0.022,0.5,NO,NO
8301,670190012055,2147012,2157003,2330001,4.88,0.0,0.0,None,0.024,0.5,NO,NO
2426,M500344106,1208344,1209005,2374002,70.4,0.0,0.0,"5/16""",0.022,0.5,NO,NO
3417,6756152,2366056,2367017,1073003,13.54,0.0,0.0,"5/16""",0.022,0.5,NO,NO
10924,SMCP24S,SMCP24S-BAND,CP58,None,45.278,6.125,6.06,None,0.025,0.625,NO,NO
5325,531300070052,4381070,4456001,4452001052,10.12,2.84,2.76,7 mm,0.024,0.356,NO,NO
945,620040500,2366040,2367024,1318001,10.37,0.0,0.0,"5/16""",0.022,0.5,NO,NO
7494,M222090175,7C09000,1151002,1278001MC,20.48,5.75,5.67,"5/16""",0.028,0.562,NO,NO


#### Build Dataframe for Band Data

In [9]:
valid_process = ['IN-LINE SINGLE NOTCH DIE', 'IN-LINE DOUBLE NOTCH DIE', 'IN-LINE BAND STAMPING', 'IN-LINE 105 NOTCH DIE', 'CUT & CURL']
df_bands = pd.DataFrame.from_records(result_band)
df_bands.columns = ['Band', 'FeedLength', 'CutOutA', 'CutOutB', 'CutOutC', 'DimA', 'DimB', 'DimC', 'Process', 'Description']
data_type_dict = {'Band' : str, 'FeedLength' : float,'CutOutA' : str,'CutOutB' : str,'CutOutC' : str,'DimA' : float,'DimB' : float,'DimC' : float,'Process' : str, 'Description' : str}
df_bands = df_bands.astype(data_type_dict)
df_bands = df_bands.convert_dtypes()

# Shape Data Set
df_bands = df_bands[1:]
df_bands['Band'] = df_bands['Band'].str.upper()
df_bands['Process'] = df_bands['Process'].str.upper()
df_bands = df_bands[df_bands['Process'].isin(valid_process)]
df_bands = df_bands[df_bands['Band'].str.contains('OBS|X|OLD|PSR|CH|NO ACTIVE') == False]
df_bands['FeedLength'] = df_bands['FeedLength'].round(3)
df_bands.fillna({'CutOutA' : 'NON-MULTI'}, inplace=True)
df_bands.fillna({'DimA' : 0.000}, inplace=True)
df_bands.fillna({'DimB' : 0.000}, inplace=True)
df_bands.fillna({'DimC' : 0.000}, inplace=True)
df_bands.drop_duplicates(subset='Band', keep='first', inplace=True)
df_bands = df_bands.dropna(subset='Band')

#df_bands.query('BandLength < 10.000', inplace=True)

df_bands.shape

(7122, 10)

#### Load Spare Parts to SQL Server Table

In [165]:
data_type_dict = {'StandardCost' : sqlalchemy.types.FLOAT, 'OnHand' : sqlalchemy.types.INT,'PartNum' : sqlalchemy.types.VARCHAR(255),'ReOrderPt' : sqlalchemy.types.INT, 'EngPartNum' : sqlalchemy.types.VARCHAR(255), 'Desc1' : sqlalchemy.types.VARCHAR(255),'Desc2' : sqlalchemy.types.VARCHAR(255),'Mfg' : sqlalchemy.types.VARCHAR(255),'MfgPn' : sqlalchemy.types.VARCHAR(255), 'Cabinet' : sqlalchemy.types.VARCHAR(255),'Drawer' : sqlalchemy.types.VARCHAR(255),'ReOrderDate' : sqlalchemy.types.VARCHAR(255),'DeptUse' : sqlalchemy.types.VARCHAR(255),'DeptPurch' : sqlalchemy.types.VARCHAR(255)}

df_spares.to_sql('tblSpares', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

28

#### Load Clamp to SQL Server Table

In [99]:
data_type_dict = {'PartNumber' : sqlalchemy.types.VARCHAR(255), 'Band' : sqlalchemy.types.VARCHAR(255),'Housing' : sqlalchemy.types.VARCHAR(255),'Screw' : sqlalchemy.types.VARCHAR(255), 'Feed' : sqlalchemy.types.Float, 'DiaMax' : sqlalchemy.types.Float,'DiaMin' : sqlalchemy.types.Float,'HexSz' : sqlalchemy.types.VARCHAR(255),'BandThickness' : sqlalchemy.types.Float, 'BandWidth' : sqlalchemy.types.Float,'CamInspect' : sqlalchemy.types.VARCHAR(255),'ScrDrvChk' : sqlalchemy.types.VARCHAR(255)}

df_clamps.to_sql('parts_clamps', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

32

#### Load Band Data to SQL Server Table

In [172]:
data_type_dict = {'Band' : sqlalchemy.types.VARCHAR(255),'FeedLength' : sqlalchemy.types.FLOAT, 'CutOutA' : sqlalchemy.types.VARCHAR(255), 'CutOutB' : sqlalchemy.types.VARCHAR(255), 'CutOutC' : sqlalchemy.types.VARCHAR(255),'DimA' : sqlalchemy.types.FLOAT,'DimB' : sqlalchemy.types.FLOAT,'DimC' : sqlalchemy.types.FLOAT, 'Process' : sqlalchemy.types.VARCHAR(255), 'Description' : sqlalchemy.types.VARCHAR(255)}

df_bands.to_sql('parts_bands', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

16